<a href="https://colab.research.google.com/github/mulumbi/Spotify_Data_Analysis/blob/main/spotify_anal_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for maths
import numpy as np
import pandas as pd
import scipy.stats as stats
import json
import random 

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
from matplotlib.pyplot import figure
import datetime as dt

% matplotlib inline

In [2]:
# loading the different datasets

streams1 = pd.read_json("/content/StreamingHistory0.json", orient='records')
streams2 = pd.read_json("/content/StreamingHistory1.json", orient='records')
streams3 = pd.read_json("/content/StreamingHistory2.json", orient='records')
stream_frames = [streams1, streams2, streams3]
streams = pd.concat(stream_frames)
streams.head()


,endTime,artistName,trackName,msPlayed
0,2021-07-18 22:11,Selena Gomez,Back To You,1582
1,2021-07-18 22:11,Lauv,Modern Loneliness,225776
2,2021-07-18 22:11,Adele,Send My Love (To Your New Lover),1951
3,2021-07-28 00:00,James Carter,Used To,159258
4,2021-07-28 00:03,James Carter,Used To,22527684


In [3]:
# converting the df from long to wide
grouped_df = streams.groupby(["trackName", "artistName"])
song_stream_times = grouped_df["msPlayed"].apply(list)
song_stream_times = song_stream_times.reset_index()



# sum up total stream minutes per song and sort in descending order
song_stream_times["msPlayed"] = song_stream_times["msPlayed"].map(lambda msPlayed: sum(msPlayed)//3600000)
cleaned_stream_times =song_stream_times.query("msPlayed > 6")
cleaned_stream_times.rename(
    columns={"msPlayed":"hrsPlayed"}
          ,inplace=True)
cleaned_stream_times.sort_values("hrsPlayed", ascending=False)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,trackName,artistName,hrsPlayed
6330,mirrorball,Taylor Swift,116
3668,Nico's Red Truck,Dijon,86
4998,Style,Taylor Swift,84
5637,Used To,James Carter,70
6232,emily,Jeremy Zucker,64
...,...,...,...
1121,Cool Again,Shoffy,7
3461,Monster,Kanye West,7
3574,NDOVU NI KUU,NDOVU KUU,7
6228,eMcimbini (Live),Kabza De Small,7


Next steps:


*   create another df grouping the individual stream dates
*   Join the two dfs to create a wide version of the long df we originally had

*   Filter out any tracks that I listened to for less than 2 hours
*   visualize my top artists and bottom artists in a time vs artist graph, where the data points are individual songs (with labels)

*   do the same with my bottom 50 artists
*   create a streaming timeline per artist using dates (per month) ie

 *  see if I binge listen to an artist on a day vs if I shuffle through multiple in a session

 *   see if theres artists I listen to everyyday/ frequently and the habits ie do I listen to some artists on weekdays vs weekends









In [4]:
# creating a list of unique artists and creation of an artist class


artist_list = list(set(cleaned_stream_times["artistName"].tolist()))
# remove podcasts from the list
artist_list.remove('The Afrobeats Podcast')
artist_list.remove('THE GOOD, THE BAD AND THE BUSHERIAN ' )


Artist Class
Members:


*   List of songs
*   List listening times
*   Total listening time

Song Class
Members:

*   Total listening time
*   List of map of day of the week to number of streams during those days
*   Average time of day I listened to the song


In [5]:
# creating artist and song class
class Artist:
    name = ""
    songs = []
    song_titles = []
    listening_times = []
    total_hrs = 0


    def __init__(self):
        pass
      
   #def __init__(self, song, time):
     # if song not in self.songs:
       # self.songs.append(song)
      #  self.listening_times.append(time)


class Song:
    title = ""
    avg_listening_time = '00:00:00'
    dates = []
    durations = []
    total_time = 0


    def __init__(self):
        pass
      
    #def __init__(self, song, time):
     # self.title = song
     # self.dates.append(time)

    def avg_time(times):
      avg = 0
      for elem in times:
          avg += elem.second + 60*elem.minute + 3600*elem.hour
      avg /= len(times)
      rez = str(avg/3600) + ' ' + str((avg%3600)/60) + ' ' + str(avg%60)
      return dt.datetime.strptime(rez, "%H %M %S")

In [6]:
# populating the Artist class
Artists = []



for musician in artist_list:
  curr_artist = Artist()
  curr_artist.name = musician
  # get all songs by that artist
  all_songs = streams.loc[streams['artistName'] == musician]
  all_songs_as_list = all_songs['trackName'].tolist()
  all_unique_songs = list(set(all_songs_as_list))
  curr_artist.song_titles = all_unique_songs
  curr_artist.songs = []
  Artists.append(curr_artist)
print(len(Artists))
print(Artists[218].name)
print(Artists[218].song_titles)

220
Bazzi
['I Like That', 'Why', 'Honest', 'Beautiful', 'Beautiful (feat. Camila Cabello)', 'Young & Alive', 'I Got You', '3:15', "Renee's Song", 'Mine']


In [9]:
# grouping songs by endTimes
grouped_end_times = grouped_df["endTime"].apply(list)
final_grouped_endtimes = grouped_end_times.reset_index()

# grouping songs by duration
grouped_durations = grouped_df["msPlayed"].apply(list)
final_grouped_durations = grouped_durations.reset_index()


for artist in Artists:
  totals = 0
  for song in artist.song_titles:
    curr_song = Song()
    curr_song.title = song
    dates_rcd = final_grouped_endtimes.loc[(final_grouped_endtimes['artistName'] == artist.name) & (final_grouped_endtimes['trackName'] == song)]
    duration_rcd = final_grouped_durations.loc[(final_grouped_durations['artistName'] == artist.name) & (final_grouped_durations['trackName'] == song)]
    total_hrs = cleaned_stream_times.loc[(cleaned_stream_times['artistName'] == artist.name) & (cleaned_stream_times['trackName'] == song)]
    curr_song.dates = dates_rcd['endTime'].tolist()
    curr_song.duration= duration_rcd['msPlayed'].tolist()
    hrs_list = total_hrs['hrsPlayed'].tolist()
    if len(hrs_list) == 0:
      curr_song.total_time = 0
    else:
      curr_song.total_time = hrs_list[0]
      
    artist.songs.append(curr_song)
    totals += int(curr_song.total_time)
    artist.songs = list(set(artist.songs))

  #update total hours for the artist
  artist.total_hrs = totals
    #print(artist.name, curr_song.title, curr_song.dates, curr_song.durations)

In [22]:
print(random.choice(Artists).name)
print(random.choice(Artists).song_titles)
sample = random.choice(Artists).songs[0]
print(sample.title)
print(sample.dates)
random_artist = random.choice(Artists)
print(random_artist.name)
print(random_artist.total_hrs)
[print(song.title) for song in random_artist.songs]


# sort artists by listening time
Artists = sorted(Artists, key=lambda artist: artist.total_hrs, reverse=True)

top_20_artists = Artists[0:20]
bottom_20_artists = Artists[len(Artists)-20:]



Reece Madlisa
['Appreciate You', 'Lay You Down Easy (feat. Sean Paul)', 'No Way No', 'Rude', 'Red Dress']
Small Things
[['2022-05-05 09:51']]
John Mayer
223
Til the Right One Comes
Message In a Bottle - Live at the Oak Mountain Amphitheater, Birmingham, AL - September 2002
New Light
Moving On and Getting Over
Slow Dancing in a Burning Room - Acoustic
Something Like Olivia
Come Back to Bed
I Guess I Just Feel Like
Shadow Days
No Such Thing
Split Screen Sadness
Something's Missing
Stop This Train
Perfectly Lonely
Say
In the Blood
My Stupid Mouth
You're Gonna Live Forever in Me
Waiting On the World to Change
Wildfire
Slow Dancing in a Burning Room
Emoji of a Wave
Who Says
Last Train Home
Carry Me Away
All We Ever Do Is Say Goodbye
XO
Born and Raised
I Will Be Found (Lost At Sea)
Half of My Heart
Clarity
Your Body Is a Wonderland
All I Want Is to Be With You
The Heart of Life
Heartbreak Warfare
Wild Blue
Love Is A Verb
Who You Love (feat. Katy Perry)
Waitin' On The Day
Gravity
Changing
In 